# Use Torchtune Llama_32_1b_lora for Classification Tasks

## 1 Imports

In [3]:
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchtune.modules import RotaryPositionalEmbeddings,MultiHeadAttention,RMSNorm, TransformerDecoder,KVCache
from torchtune.models.llama3 import llama3_tokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
# for pre-trained weights
from safetensors.torch import load
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import os
from pathlib import Path

os.getcwd()


import error: No module named 'triton'
cpu


'c:\\Users\\MRM\\Desktop\\Data_Analytics\\Medium_and_PPB\\Machine_Learning\\Machine_Learning_Projects\\NLP\\Build_a_LLM_from_scratch\\torchtune'

In [ ]:
tokenizer = llama3_tokenizer(
    path=os.path.join(os.getcwd() + "/Llama_32_1B/original/tokenizer.model")
    )
print(tokenizer.special_tokens)

In [4]:
# Calling a transformer encoder from torchtune with 1B
from torchtune.models.llama3_2 import lora_llama3_2_1b, llama3_2_1b
# Initialize the model architecture
base_model = llama3_2_1b()
base_model



TransformerDecoder(
  (tok_embeddings): Embedding(128256, 2048)
  (layers): ModuleList(
    (0-15): 16 x TransformerSelfAttentionLayer(
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (output_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (pos_embeddings): Llama3ScaledRoPE()
      )
      (mlp): FeedForward(
        (w1): Linear(in_features=2048, out_features=8192, bias=False)
        (w2): Linear(in_features=8192, out_features=2048, bias=False)
        (w3): Linear(in_features=2048, out_features=8192, bias=False)
        (activation): SiLU()
      )
      (sa_norm): RMSNorm()
      (mlp_norm): RMSNorm()
      (sa_scale): Identity()
      (mlp_scale): Identity()
    )
  )
  (norm): RMSNorm()
)

In [ ]:
#  ["q_proj", "k_proj", "v_proj", "output_proj"]
latt_mods = ["q_proj", "v_proj"]
lora_llama32_1b = lora_llama3_2_1b(lora_attn_modules=latt_mods)
lora_llama32_1b

In [ ]:
total_params = sum(p.numel() for p in lora_llama32_1b.parameters())
print(f"Total number of parameters: {total_params:,}")

In [ ]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Calculate total number of elements per parameter
        param_size = param.numel()
        total_params += param_size
        # Check if gradients are stored for this parameter
        if param.requires_grad:
            total_grads += param_size

    # Calculate buffer size (non-parameters that require memory)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Size in bytes = (Number of elements) * (Size of each element in bytes)
    # We assume parameters and gradients are stored in the same type as input dtype
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Convert bytes to gigabytes
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print('lora_llama32_1b memory footprint')
print(f"float32 (PyTorch default): {model_memory_size(lora_llama32_1b, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(lora_llama32_1b, input_dtype=torch.bfloat16):.2f} GB")
print('')
print('base llama32_1b memory footprint')
print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")



In [11]:
from safetensors.torch import load_file
file_path ='.\Llama_32_1B\model.safetensors'
pretrained_weights = load_file(file_path)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\MRM\AppData\Local\Temp\ipykernel_9180\38733429.py:2: SyntaxWarning: invalid escape sequence '\L'
  file_path ='.\Llama_32_1B\model.safetensors'


In [ ]:
# Step 3: Load the weights into the model
model_state_dict = model.state_dict()
for name, param in model_state_dict.items():
    print(name)

In [ ]:

# Step 3: Load the weights into the model
model_state_dict = model.state_dict()
for name, param in model_state_dict.items():
    if name in weights:
        model_state_dict[name].copy_(weights[name])
    else:
        print(f"Warning: {name} not found in the safetensors file.")


In [ ]:
# Assuming that base_model already has the pretrained Llama32 weights,
# this will directly load them into your LoRA model without any conversion necessary.
#lora_llama32_1b.load_state_dict(model.state_dict(), strict=False)

In [22]:
base_model

TransformerDecoder(
  (tok_embeddings): Embedding(128256, 2048)
  (layers): ModuleList(
    (0-15): 16 x TransformerSelfAttentionLayer(
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (output_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (pos_embeddings): Llama3ScaledRoPE()
      )
      (mlp): FeedForward(
        (w1): Linear(in_features=2048, out_features=8192, bias=False)
        (w2): Linear(in_features=8192, out_features=2048, bias=False)
        (w3): Linear(in_features=2048, out_features=8192, bias=False)
        (activation): SiLU()
      )
      (sa_norm): RMSNorm()
      (mlp_norm): RMSNorm()
      (sa_scale): Identity()
      (mlp_scale): Identity()
    )
  )
  (norm): RMSNorm()
)

In [ ]:

# Step 3: Inspect the state dictionaries
print("Pre-trained model state dict keys:")

for name, param in pretrained_weights.items():
    print(name)

print("\nCustom model state dict keys:")
for name, param in base_model.named_parameters():
    print(name)


Pre-trained model state dict keys:
model.embed_tokens.weight
model.layers.0.input_layernorm.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.post_attention_layernorm.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.post_attention_layernorm.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.10.input_layernorm.weight
model.layers.10.mlp.down_proj.weight
model.layers.10.mlp.gate_proj.weight
model.layers.10.mlp.up_proj.weight
model.layers.10.post_attention_layernorm.weight
model.layers.10.self_attn.k_proj.weight
model.lay

In [17]:

# Step 4: Create a mapping between the keys

mapping = {
    'model.embed_tokens.weight':'tok_embeddings.weight',
    'model.layers.0.input_layernorm.weight':'layers.0.sa_norm.scale', #Self-att norm
    'model.layers.0.mlp.down_proj.weight':'layers.0.mlp.w2.weight',
    'model.layers.0.mlp.gate_proj.weight':'layers.0.mlp.w1.weight',
    'model.layers.0.mlp.up_proj.weight':'layers.0.mlp.w3.weight',
    'model.layers.0.post_attention_layernorm.weight':'layers.0.mlp_norm.scale',
    'model.layers.0.self_attn.k_proj.weight':'layers.0.attn.k_proj.weight',
    'model.layers.0.self_attn.o_proj.weight':'layers.0.attn.output_proj.weight',
    'model.layers.0.self_attn.q_proj.weight':'layers.0.attn.q_proj.weight',
    'model.layers.0.self_attn.v_proj.weight':'layers.0.attn.v_proj.weight',
    'model.layers.1.input_layernorm.weight':'layers.1.sa_norm.scale', #Self-att norm
    'model.layers.1.mlp.down_proj.weight':'layers.1.mlp.w2.weight',
    'model.layers.1.mlp.gate_proj.weight':'layers.1.mlp.w1.weight',
    'model.layers.1.mlp.up_proj.weight':'layers.1.mlp.w3.weight',
    'model.layers.1.post_attention_layernorm.weight':'layers.1.mlp_norm.scale',
    'model.layers.1.self_attn.k_proj.weight':'layers.1.attn.k_proj.weight',
    'model.layers.1.self_attn.o_proj.weight':'layers.1.attn.output_proj.weight',
    'model.layers.1.self_attn.q_proj.weight':'layers.1.attn.q_proj.weight',
    'model.layers.1.self_attn.v_proj.weight':'layers.1.attn.v_proj.weight',
    'model.layers.2.input_layernorm.weight':'layers.2.sa_norm.scale', #Self-att norm
    'model.layers.2.mlp.down_proj.weight':'layers.2.mlp.w2.weight',
    'model.layers.2.mlp.gate_proj.weight':'layers.2.mlp.w1.weight',
    'model.layers.2.mlp.up_proj.weight':'layers.2.mlp.w3.weight',
    'model.layers.2.post_attention_layernorm.weight':'layers.2.mlp_norm.scale',
    'model.layers.2.self_attn.k_proj.weight':'layers.2.attn.k_proj.weight',
    'model.layers.2.self_attn.o_proj.weight':'layers.2.attn.output_proj.weight',
    'model.layers.2.self_attn.q_proj.weight':'layers.2.attn.q_proj.weight',
    'model.layers.2.self_attn.v_proj.weight':'layers.2.attn.v_proj.weight',
    'model.layers.3.input_layernorm.weight':'layers.3.sa_norm.scale', #Self-att norm
    'model.layers.3.mlp.down_proj.weight':'layers.3.mlp.w2.weight',
    'model.layers.3.mlp.gate_proj.weight':'layers.3.mlp.w1.weight',
    'model.layers.3.mlp.up_proj.weight':'layers.3.mlp.w3.weight',
    'model.layers.3.post_attention_layernorm.weight':'layers.3.mlp_norm.scale',
    'model.layers.3.self_attn.k_proj.weight':'layers.3.attn.k_proj.weight',
    'model.layers.3.self_attn.o_proj.weight':'layers.3.attn.output_proj.weight',
    'model.layers.3.self_attn.q_proj.weight':'layers.3.attn.q_proj.weight',
    'model.layers.3.self_attn.v_proj.weight':'layers.3.attn.v_proj.weight',
    'model.layers.4.input_layernorm.weight':'layers.4.sa_norm.scale', #Self-att norm
    'model.layers.4.mlp.down_proj.weight':'layers.4.mlp.w2.weight',
    'model.layers.4.mlp.gate_proj.weight':'layers.4.mlp.w1.weight',
    'model.layers.4.mlp.up_proj.weight':'layers.4.mlp.w3.weight',
    'model.layers.4.post_attention_layernorm.weight':'layers.4.mlp_norm.scale',
    'model.layers.4.self_attn.k_proj.weight':'layers.4.attn.k_proj.weight',
    'model.layers.4.self_attn.o_proj.weight':'layers.4.attn.output_proj.weight',
    'model.layers.4.self_attn.q_proj.weight':'layers.4.attn.q_proj.weight',
    'model.layers.4.self_attn.v_proj.weight':'layers.4.attn.v_proj.weight',
    'model.layers.5.input_layernorm.weight':'layers.5.sa_norm.scale', #Self-att norm
    'model.layers.5.mlp.down_proj.weight':'layers.5.mlp.w2.weight',
    'model.layers.5.mlp.gate_proj.weight':'layers.5.mlp.w1.weight',
    'model.layers.5.mlp.up_proj.weight':'layers.5.mlp.w3.weight',
    'model.layers.5.post_attention_layernorm.weight':'layers.5.mlp_norm.scale',
    'model.layers.5.self_attn.k_proj.weight':'layers.5.attn.k_proj.weight',
    'model.layers.5.self_attn.o_proj.weight':'layers.5.attn.output_proj.weight',
    'model.layers.5.self_attn.q_proj.weight':'layers.5.attn.q_proj.weight',
    'model.layers.5.self_attn.v_proj.weight':'layers.5.attn.v_proj.weight',
    'model.layers.6.input_layernorm.weight':'layers.6.sa_norm.scale', #Self-att norm
    'model.layers.6.mlp.down_proj.weight':'layers.6.mlp.w2.weight',
    'model.layers.6.mlp.gate_proj.weight':'layers.6.mlp.w1.weight',
    'model.layers.6.mlp.up_proj.weight':'layers.6.mlp.w3.weight',
    'model.layers.6.post_attention_layernorm.weight':'layers.6.mlp_norm.scale',
    'model.layers.6.self_attn.k_proj.weight':'layers.6.attn.k_proj.weight',
    'model.layers.6.self_attn.o_proj.weight':'layers.6.attn.output_proj.weight',
    'model.layers.6.self_attn.q_proj.weight':'layers.6.attn.q_proj.weight',
    'model.layers.6.self_attn.v_proj.weight':'layers.6.attn.v_proj.weight',
    'model.layers.7.input_layernorm.weight':'layers.7.sa_norm.scale', #Self-att norm
    'model.layers.7.mlp.down_proj.weight':'layers.7.mlp.w2.weight',
    'model.layers.7.mlp.gate_proj.weight':'layers.7.mlp.w1.weight',
    'model.layers.7.mlp.up_proj.weight':'layers.7.mlp.w3.weight',
    'model.layers.7.post_attention_layernorm.weight':'layers.7.mlp_norm.scale',
    'model.layers.7.self_attn.k_proj.weight':'layers.7.attn.k_proj.weight',
    'model.layers.7.self_attn.o_proj.weight':'layers.7.attn.output_proj.weight',
    'model.layers.7.self_attn.q_proj.weight':'layers.7.attn.q_proj.weight',
    'model.layers.7.self_attn.v_proj.weight':'layers.7.attn.v_proj.weight',
    'model.layers.8.input_layernorm.weight':'layers.8.sa_norm.scale', #Self-att norm
    'model.layers.8.mlp.down_proj.weight':'layers.8.mlp.w2.weight',
    'model.layers.8.mlp.gate_proj.weight':'layers.8.mlp.w1.weight',
    'model.layers.8.mlp.up_proj.weight':'layers.8.mlp.w3.weight',
    'model.layers.8.post_attention_layernorm.weight':'layers.8.mlp_norm.scale',
    'model.layers.8.self_attn.k_proj.weight':'layers.8.attn.k_proj.weight',
    'model.layers.8.self_attn.o_proj.weight':'layers.8.attn.output_proj.weight',
    'model.layers.8.self_attn.q_proj.weight':'layers.8.attn.q_proj.weight',
    'model.layers.8.self_attn.v_proj.weight':'layers.8.attn.v_proj.weight',
    'model.layers.9.input_layernorm.weight':'layers.9.sa_norm.scale', #Self-att norm
    'model.layers.9.mlp.down_proj.weight':'layers.9.mlp.w2.weight',
    'model.layers.9.mlp.gate_proj.weight':'layers.9.mlp.w1.weight',
    'model.layers.9.mlp.up_proj.weight':'layers.9.mlp.w3.weight',
    'model.layers.9.post_attention_layernorm.weight':'layers.9.mlp_norm.scale',
    'model.layers.9.self_attn.k_proj.weight':'layers.9attn.k_proj.weight',
    'model.layers.9.self_attn.o_proj.weight':'layers.9attn.output_proj.weight',
    'model.layers.9.self_attn.q_proj.weight':'layers.9attn.q_proj.weight',
    'model.layers.9.self_attn.v_proj.weight':'layers.9attn.v_proj.weight',
    'model.layers.10.input_layernorm.weight':'layers.9.sa_norm.scale', #Self-att norm
    'model.layers.10.mlp.down_proj.weight':'layers.10.mlp.w2.weight',
    'model.layers.10.mlp.gate_proj.weight':'layers.10.mlp.w1.weight',
    'model.layers.10.mlp.up_proj.weight':'layers.10.mlp.w3.weight',
    'model.layers.10.post_attention_layernorm.weight':'layers.10.mlp_norm.scale',
    'model.layers.10.self_attn.k_proj.weight':'layers.10.attn.k_proj.weight',
    'model.layers.10.self_attn.o_proj.weight':'layers.10.attn.output_proj.weight',
    'model.layers.10.self_attn.q_proj.weight':'layers.10.attn.q_proj.weight',
    'model.layers.10.self_attn.v_proj.weight':'layers.10.attn.v_proj.weight',
    'model.layers.11.input_layernorm.weight':'layers.11.sa_norm.scale', #Self-att norm
    'model.layers.11.mlp.down_proj.weight':'layers.11.mlp.w2.weight',
    'model.layers.11.mlp.gate_proj.weight':'layers.11.mlp.w1.weight',
    'model.layers.11.mlp.up_proj.weight':'layers.11.mlp.w3.weight',
    'model.layers.11.post_attention_layernorm.weight':'layers.11.mlp_norm.scale',
    'model.layers.11.self_attn.k_proj.weight':'layers.11.attn.k_proj.weight',
    'model.layers.11.self_attn.o_proj.weight':'layers.11.attn.output_proj.weight',
    'model.layers.11.self_attn.q_proj.weight':'layers.11.attn.q_proj.weight',
    'model.layers.11.self_attn.v_proj.weight':'layers.11.attn.v_proj.weight',
    'model.layers.12.input_layernorm.weight':'layers.12.sa_norm.scale', #Self-att norm
    'model.layers.12.mlp.down_proj.weight':'layers.12.mlp.w2.weight',
    'model.layers.12.mlp.gate_proj.weight':'layers.12.mlp.w1.weight',
    'model.layers.12.mlp.up_proj.weight':'layers.12.mlp.w3.weight',
    'model.layers.12.post_attention_layernorm.weight':'layers.12.mlp_norm.scale',
    'model.layers.12.self_attn.k_proj.weight':'layers.12.attn.k_proj.weight',
    'model.layers.12.self_attn.o_proj.weight':'layers.12.attn.output_proj.weight',
    'model.layers.12.self_attn.q_proj.weight':'layers.12.attn.q_proj.weight',
    'model.layers.12.self_attn.v_proj.weight':'layers.12.attn.v_proj.weight',
    'model.layers.13.input_layernorm.weight':'layers.13.sa_norm.scale', #Self-att norm
    'model.layers.13.mlp.down_proj.weight':'layers.13.mlp.w2.weight',
    'model.layers.13.mlp.gate_proj.weight':'layers.13.mlp.w1.weight',
    'model.layers.13.mlp.up_proj.weight':'layers.13.mlp.w3.weight',
    'model.layers.13.post_attention_layernorm.weight':'layers.13.mlp_norm.scale',
    'model.layers.13.self_attn.k_proj.weight':'layers.13.attn.k_proj.weight',
    'model.layers.13.self_attn.o_proj.weight':'layers.13.attn.output_proj.weight',
    'model.layers.13.self_attn.q_proj.weight':'layers.13.attn.q_proj.weight',
    'model.layers.13.self_attn.v_proj.weight':'layers.13.attn.v_proj.weight',
    'model.layers.14.input_layernorm.weight':'layers.14.sa_norm.scale', #Self-att norm
    'model.layers.14.mlp.down_proj.weight':'layers.14.mlp.w2.weight',
    'model.layers.14.mlp.gate_proj.weight':'layers.14.mlp.w1.weight',
    'model.layers.14.mlp.up_proj.weight':'layers.14.mlp.w3.weight',
    'model.layers.14.post_attention_layernorm.weight':'layers.14.mlp_norm.scale',
    'model.layers.14.self_attn.k_proj.weight':'layers.14.attn.k_proj.weight',
    'model.layers.14.self_attn.o_proj.weight':'layers.14.attn.output_proj.weight',
    'model.layers.14.self_attn.q_proj.weight':'layers.14.attn.q_proj.weight',
    'model.layers.14.self_attn.v_proj.weight':'layers.14.attn.v_proj.weight',
    'model.layers.15.input_layernorm.weight':'layers.15.sa_norm.scale', #Self-att norm
    'model.layers.15.mlp.down_proj.weight':'layers.15.mlp.w2.weight',
    'model.layers.15.mlp.gate_proj.weight':'layers.15.mlp.w1.weight',
    'model.layers.15.mlp.up_proj.weight':'layers.15.mlp.w3.weight',
    'model.layers.15.post_attention_layernorm.weight':'layers.15.mlp_norm.scale',
    'model.layers.15.self_attn.k_proj.weight':'layers.15.attn.k_proj.weight',
    'model.layers.15.self_attn.o_proj.weight':'layers.15.attn.output_proj.weight',
    'model.layers.15.self_attn.q_proj.weight':'layers.15.attn.q_proj.weight',
    'model.layers.15.self_attn.v_proj.weight':'layers.15.attn.v_proj.weight',
    'model.norm.weight':'norm.scale'
    
    # Add mappings for other layers and parameters as needed
}


# Step 5: Load the weights manually using the mapping
custom_state_dict = base_model.state_dict()
for pretrained_key, custom_key in mapping.items():
    if pretrained_key in pretrained_weights and custom_key in custom_state_dict:
        custom_state_dict[custom_key] = pretrained_weights[pretrained_key]
base_model.load_state_dict(custom_state_dict)

# Step 6: Verify that the weights are loaded correctly
for name, param in base_model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

# Now your custom model should have the weights from the pre-trained modelmodel.embed_tokens.weight


tok_embeddings.weight tensor([[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
        [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
        [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
        ...,
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066]])
layers.0.attn.q_proj.weight tensor([[-0.0183,  0.0071,  0.0219,  ..., -0.0070, -0.0089,  0.0149],
        [ 0.0112,  0.0593,  0.0630,  ..., -0.0334, -0.0148,  0.0058],
        [ 0.0182,  0.0141,  0.0361,  ..., -0.0432, -0.0388, -0.0233],
        ...,
        [ 0.0305,  0.0289,  0.0801,  ..., -0.0767, -0.0311, -0.0334],
        [ 0.0242, -0.0325,  0.0369,  ..., -0.0123, -0.0269, -0.0151],
        [-0.0264, -0.0498, -0.0210,  ...,  0.0601,  0.0130, -0.0007]])
layers.0.attn.k_proj.weight tensor([[ 0.0559,  0.1123,  0.0718,  ..., -0.1045, -0.

In [ ]:
def assign(left, right, tensor_name="unknown"):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch in tensor '{tensor_name}'. Left: {left.shape}, Right: {right.shape}")

    if isinstance(right, torch.Tensor):
        return torch.nn.Parameter(right.clone().detach())
    else:
        return torch.nn.Parameter(torch.tensor(right))


def load_weights_into_llama(torchtune_model, param_config, params):
    # Function to adapt HF downloaded weights to a torchtune Llama3.2_1B encoder sample
    # Adapted from S. Raschka´s book "Building a LLM from Scratch"
    
    tok_embeddings.weight = assign(torchtune_model.tok_embeddings.weight, params["model.embed_tokens.weight"], "model.embed_tokens.weight")
    'model.layers.0.input_layernorm.weight':'layers.0.sa_norm.scale', #Self-att norm
    'model.layers.0.mlp.down_proj.weight':'layers.0.mlp.w2.weight',
    'model.layers.0.mlp.gate_proj.weight':'layers.0.mlp.w1.weight',
    'model.layers.0.mlp.up_proj.weight':'layers.0.mlp.w3.weight',
    'model.layers.0.post_attention_layernorm.weight':'layers.0.mlp_norm.scale',
    'model.layers.0.self_attn.k_proj.weight':'layers.0.attn.k_proj.weight',
    'model.layers.0.self_attn.o_proj.weight':'layers.0.attn.output_proj.weight',
    'model.layers.0.self_attn.q_proj.weight':'layers.0.attn.q_proj.weight',
    'model.layers.0.self_attn.v_proj.weight':'layers.0.attn.v_proj.weight',
    for l in range(["layers"]):

        # Load attention weights
        model.trf_blocks[l].att.W_query.weight = assign(model.trf_blocks[l].att.W_query.weight,
                                                        params[f"model.layers.{l}.self_attn.q_proj.weight"],
                                                        f"model.layers.{l}.self_attn.q_proj.weight"
                                                        )
        model.trf_blocks[l].att.W_key.weight = assign(
            model.trf_blocks[l].att.W_key.weight,
            params[f"model.layers.{l}.self_attn.k_proj.weight"],
            f"model.layers.{l}.self_attn.k_proj.weight"
        )
        model.trf_blocks[l].att.W_value.weight = assign(
            model.trf_blocks[l].att.W_value.weight,
            params[f"model.layers.{l}.self_attn.v_proj.weight"],
            f"model.layers.{l}.self_attn.v_proj.weight"
        )
        model.trf_blocks[l].att.out_proj.weight = assign(
            model.trf_blocks[l].att.out_proj.weight,
            params[f"model.layers.{l}.self_attn.o_proj.weight"],
            f"model.layers.{l}.self_attn.o_proj.weight"
        )
        model.trf_blocks[l].norm1.weight = assign(
            model.trf_blocks[l].norm1.weight,
            params[f"model.layers.{l}.input_layernorm.weight"],
            f"model.layers.{l}.input_layernorm.weight"
        )

        # Load FeedForward weights
        model.trf_blocks[l].ff.fc1.weight = assign(
            model.trf_blocks[l].ff.fc1.weight,
            params[f"model.layers.{l}.mlp.gate_proj.weight"],
            f"model.layers.{l}.mlp.gate_proj.weight"
        )
        model.trf_blocks[l].ff.fc2.weight = assign(
            model.trf_blocks[l].ff.fc2.weight,
            params[f"model.layers.{l}.mlp.up_proj.weight"],
            f"model.layers.{l}.mlp.up_proj.weight"
        )
        model.trf_blocks[l].ff.fc3.weight = assign(
            model.trf_blocks[l].ff.fc3.weight,
            params[f"model.layers.{l}.mlp.down_proj.weight"],
            f"model.layers.{l}.mlp.down_proj.weight"
        )
        model.trf_blocks[l].norm2.weight = assign(
            model.trf_blocks[l].norm2.weight,
            params[f"model.layers.{l}.post_attention_layernorm.weight"],
            f"model.layers.{l}.post_attention_layernorm.weight"
        )

    # Load output layer weights
    model.final_norm.weight = assign(model.final_norm.weight, params["model.norm.weight"], "model.norm.weight")

    if "lm_head.weight" in params.keys():
        model.out_head.weight = assign(model.out_head.weight, params["lm_head.weight"], "lm_head.weight")
    else:
        model.out_head.weight = assign(model.out_head.weight, params["model.embed_tokens.weight"], "model.embed_tokens.weight")
        print("Model uses weight tying.")